# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [2]:
pip install h5py

In [3]:
pip install git+https://github.com/pytorch/tnt.git@master

  Cloning https://github.com/pytorch/tnt.git (to revision master) to /tmp/pip-req-build-wfxpnhl2
  Running command git clone -q https://github.com/pytorch/tnt.git /tmp/pip-req-build-wfxpnhl2
     |████████████████████████████████| 686kB 8.9MB/s 
     |████████████████████████████████| 204kB 46.6MB/s 
  Created wheel for torchnet: filename=torchnet-0.0.5.1-cp36-none-any.whl size=30917 sha256=c2351b6746c73226532d04810161f600b5f19a19ba73966f2834a756b828309e
  Stored in directory: /tmp/pip-ephem-wheel-cache-oryyzohg/wheels/17/05/ec/d05d051a225871af52bf504f5e8daf57704811b3c1850d0012
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=96219feb61c09858f4b2a18c2af0938a6389703d8f642ce355fc21e64f49c4e1
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=9acc4ec8721cd5f75d63e7a961c8245aef46471ab512ee9881add3330cf7363

In [4]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: False


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25). 

For your convenience, we provide the dataloader for you.


In [0]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        data_path = '/content/drive/My Drive/Question3/data'
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('/content/drive/My Drive/Question3/data', subset='train')
valSet = Dataset('/content/drive/My Drive/Question3/data', subset='val')
tstSet = Dataset('/content/drive/My Drive/Question3/data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [0]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75,100)
        self.recurrent_layer = nn.LSTM(100,100,1)
        self.classify_layer = nn.Linear(100,10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [0]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()

In [15]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data)
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 20
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.3052,  Accuracy: 8.15
val Epoch: 0  , Loss: 2.3057,  Accuracy: 7.75
train Epoch: 1  , Loss: 2.3046,  Accuracy: 8.60
train Epoch: 2  , Loss: 2.3045,  Accuracy: 7.90
train Epoch: 3  , Loss: 2.3038,  Accuracy: 8.25
train Epoch: 4  , Loss: 2.3037,  Accuracy: 8.20
train Epoch: 5  , Loss: 2.3040,  Accuracy: 8.65
val Epoch: 5  , Loss: 2.3041,  Accuracy: 8.75
train Epoch: 6  , Loss: 2.3037,  Accuracy: 9.10
train Epoch: 7  , Loss: 2.3034,  Accuracy: 8.95
train Epoch: 8  , Loss: 2.3034,  Accuracy: 8.60
train Epoch: 9  , Loss: 2.3032,  Accuracy: 8.70
train Epoch: 10  , Loss: 2.3031,  Accuracy: 9.65
val Epoch: 10  , Loss: 2.3029,  Accuracy: 10.00
train Epoch: 11  , Loss: 2.3025,  Accuracy: 9.50
train Epoch: 12  , Loss: 2.3027,  Accuracy: 9.25
train Epoch: 13  , Loss: 2.3024,  Accuracy: 9.20
train Epoch: 14  , Loss: 2.3022,  Accuracy: 10.25
train Epoch: 15  , Loss: 2.3023,  Accuracy: 10.20
val Epoch: 15  , Loss: 2.3021,  Accuracy: 9.75
train Epoch: 16  , Loss: 2.3018,  Acc

## Submit your results 

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set. 


### Testing the model and reporting the results
Test the model on the testing set and save the results as a .csv file. 
submit the results.csv file generated by predict_on_test(). Also mention the best performance on the Validation set, and submit the corresponding results csv file which results in the best performance. 
################ 3rd To Do  (15 points) ###############


In [0]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)   

In [0]:
class SeqCl(nn.Module):
    def __init__(self):
        super(SeqCl, self).__init__()
        self.layer= nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=2, stride=1,padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 32, kernel_size=2, stride=1,padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=2, stride=1,padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=2, stride=1,padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout2d(p=0.2),
            Flatten(),
            nn.Linear(5760, 100),
            nn.BatchNorm1d(100),
        )
        
        self.recurrent_layer = nn.LSTM(100,100,1)
        self.classify_layer = nn.Linear(100,10)
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs = self.layer(input.view(50, 1, 15, 75))
        rnn_outputs, (hn, cn) = self.recurrent_layer(rnn_outputs.view(50, 1, 100))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SeqCl()
from torch import optim
loss_fn =  nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(),lr=1e-4)

In [50]:
num_epochs = 50
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None) 

train Epoch: 0  , Loss: 2.1618,  Accuracy: 29.55
val Epoch: 0  , Loss: 2.0196,  Accuracy: 42.50
train Epoch: 1  , Loss: 1.8923,  Accuracy: 44.40
train Epoch: 2  , Loss: 1.7166,  Accuracy: 51.35
train Epoch: 3  , Loss: 1.5823,  Accuracy: 56.65
train Epoch: 4  , Loss: 1.4728,  Accuracy: 60.55
train Epoch: 5  , Loss: 1.3698,  Accuracy: 64.25
val Epoch: 5  , Loss: 1.3362,  Accuracy: 62.50
train Epoch: 6  , Loss: 1.2733,  Accuracy: 67.65
train Epoch: 7  , Loss: 1.1961,  Accuracy: 69.75
train Epoch: 8  , Loss: 1.1151,  Accuracy: 72.00
train Epoch: 9  , Loss: 1.0474,  Accuracy: 73.60
train Epoch: 10  , Loss: 0.9878,  Accuracy: 74.50
val Epoch: 10  , Loss: 0.9930,  Accuracy: 72.50
train Epoch: 11  , Loss: 0.9442,  Accuracy: 74.80
train Epoch: 12  , Loss: 0.8808,  Accuracy: 76.10
train Epoch: 13  , Loss: 0.8352,  Accuracy: 77.55
train Epoch: 14  , Loss: 0.7941,  Accuracy: 79.15
train Epoch: 15  , Loss: 0.7488,  Accuracy: 79.85
val Epoch: 15  , Loss: 0.8267,  Accuracy: 74.00
train Epoch: 16  , L

In [51]:
# Use your best model to generate results on test set and validation set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)
print(count)

500


## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epoches) and any visualizations or graphs that you make in the process of training and evaluating your network.



I have tried to implement the following architecture based on 
 [conv-relu-pool]xN -> [affine]xM

 So here the N = 4 and M =1.
 This model has been used as the accoracy I got through it is 80%.

The architecture used is:
1. Started with the BAtch Normalisation of the input.
2. 2x2 Convolutional Layer with 8 filters with stride of 1 and pixel padding of 3.
3. ReLU Activation Layer
4. 2x2 Max Pooling layer with a stride of 2.
5. 2x2 Convolutional Layer with 32 filters with stride of 1 and pixel padding of 3.
6. ReLU Activation Layer
7. 2x2 Max Pooling layer with a stride of 2
8. 4x4 Convolutional Layer with 32 filters with stride of 1 and pixel padding of 3.
9. ReLU Activation Layer
10. 2x2 Max Pooling layer with a stride of 2
11. 4x4 Convolutional Layer with 32 filters with stride of 2 and pixel padding of 3.
12. ReLU Activation Layer
13. 2x2 Max Pooling layer with a stride of 2
14. Do the Batch Normalization before the start of each Convolutional Layer.
15. Dropout Layer with a probability of 0.2.
16. Flatten the feature map.
17. Affine layer to map input units to 10 outputs.

When I increased the epoch I got a good accuracy. As the model was able to train fast I could run it for 50 epochs.So the accuracy increased from mere 67 % to more than 90%. 

I used 2 optimizers. RmSProp and Adam Optimizer. I checked for 10 epoches and I saw that RmsProp gave me a good accuracy than Adam Optimizer.






